In [ ]:
import segmentation_models as sm
import tensorflow as tf
import numpy as np
import glob
import os
import cv2


# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import data_preparation_cnn

In [ ]:
def get_images(class_id, size_x, size_y):
    images = []
    path_suffix = 'c' + str(class_id) + '/'

    for directory_path in glob.glob('data/segmentation/test/' + path_suffix):
        for img_path in sorted(glob.glob(os.path.join(directory_path, "*.jpg"))):
            #print(img_path)
            #break
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
            img = cv2.resize(img, (size_y, size_x))
            # img = img/255
            #print(img)
            #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            images.append(img)
            #train_labels.append(label)
    #Convert list to array for machine learning processing        
    images = np.array(images)
    
    return images

In [ ]:
def get_masks(class_id, size_x, size_y):
    images = []
    path_suffix = 'c' + str(class_id) + '/'

    for directory_path in glob.glob('data/segmentation/test_mask/' + path_suffix):
        for img_path in sorted(glob.glob(os.path.join(directory_path, "*.png"))):
            #print(img_path)
            #break
            img = cv2.imread(img_path, 0)       
            img = cv2.resize(img, (size_y, size_x))
            #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            images.append(img)
            #train_labels.append(label)
    #Convert list to array for machine learning processing        
    images = np.array(images)
    
    return images

In [ ]:
def prepare_input_variables(class_id, train_images, train_masks, size_x, size_y):
    # get preprocessing for `EfficientNetB5`
    preprocess_input = sm.get_preprocessing('efficientnetb5')
    
    # prepare variables
    x_train = preprocess_input(train_images)
    y_train = np.expand_dims(train_masks, axis=3)
    # y_train = y_train/255

    x_val = get_images(class_id=class_id, size_x=size_x, size_y=size_y)
    x_val = preprocess_input(x_val)
    y_val = get_masks(class_id=class_id, size_x=size_x, size_y=size_y)
    y_val = np.expand_dims(y_val, axis=3) #May not be necessary.. leftover from previous code 
    # y_val = y_val / 255

    return x_train, y_train, x_val, y_val

In [ ]:
def build_compiled_model(size_x, size_y, metric_for_model):
    # set the correct framework for the model to work
    sm.set_framework('tf.keras')
    sm.framework()
    
    # define model
    model = sm.Unet('efficientnetb5',
                    input_shape=(size_x, size_y, 3),
                    classes=1,
                    activation='sigmoid',
                    encoder_weights='imagenet',
                    encoder_freeze=True
                   )
    if metric_for_model == 'dice':
        model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coef])
    else:
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[metric_for_model])
    
    return model

In [ ]:
def get_history_from_mask_model(class_id, size_x, size_y, metric_for_model, epochs):
    # load images and masks as input for model
    train_images, train_masks = data_preparation_cnn.get_resized_image_and_mask_lists(class_id=class_id, 
                                                                              size_x=size_x, 
                                                                              size_y=size_y)

    # build input variables for model
    x_train, y_train, x_val, y_val = prepare_input_variables(class_id, 
                                                             train_images, 
                                                             train_masks, 
                                                             size_x, 
                                                             size_y)
        
    # create a compiled model
    model = build_compiled_model(size_x, size_y, metric_for_model)
    
    print(f'beginning training with masks')
    # fit the model
    history = model.fit(x_train, 
                        y_train,
                        batch_size=32, 
                        epochs=epochs,
                        verbose=True,
                        validation_data=(x_val, y_val)
                       )
    
    # predictions = model.predict(x_val)
    
    # save model
    model_name = 'models/class' + str(class_id) + '_mask_generator.h5'
    model.save(model_name)
    
    return history

In [ ]:
from keras.losses import binary_crossentropy

In [ ]:
def dice_coef(y_true,y_pred):
    y_true_f=tf.reshape(tf.dtypes.cast(y_true, tf.float32),[-1])
    y_pred_f=tf.reshape(tf.dtypes.cast(y_pred, tf.float32),[-1])
    intersection=tf.reduce_sum(y_true_f*y_pred_f)
    return (2. * intersection + 1.) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + 1.)
def dice_loss(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    return (1-dice_coef(y_true, y_pred))
def bce_dice_loss(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    return binary_crossentropy(y_true, y_pred) + (1-dice_coef(y_true, y_pred))

In [ ]:
def build_mask_generating_models(size_x, size_y, metric_for_model, epochs):
    
    losses = []
    losses_val = []
    
    metric_results = []
    metric_results_val = []
    
    predictions = {}
    models = {}
    
    for class_id in [1,2,3,4]:
        print(f'building model for defect class {class_id}...')
        
        
        # extract the history for each model
        history = get_history_from_mask_model(class_id, size_x, size_y, metric_for_model, epochs)
        
        losses.append(history.history['loss'])
        losses_val.append(history.history['val_loss'])
        if metric_for_model == 'dice':
            pass
        else:
            metric_results = history.history[metric_for_model]
            metric_results_val = history.history['val_' + metric_for_model]
        
        # predictions[str(class_id)] = pred
        
        print('model successfully generated and saved to file!')
        print('-----'*10)
        print()
    
    return predictions
        

In [ ]:

# PARAMETERS
SIZE_X = 128
SIZE_Y = 256
EPOCHS = 10

In [ ]:
predictions = build_mask_generating_models(SIZE_X, SIZE_Y, 'dice', EPOCHS)

In [ ]:
predictions['1'].shape

In [ ]:
import keras
import matplotlib.pyplot as plt

SIZE_X = 128
SIZE_Y = 256

In [ ]:
model1 = keras.models.load_model('models/class1_mask_generator.h5', compile=False)
model2 = keras.models.load_model('models/class2_mask_generator.h5', compile=False)
model3 = keras.models.load_model('models/class3_mask_generator.h5', compile=False)
model4 = keras.models.load_model('models/class4_mask_generator.h5', compile=False)

In [ ]:
model1.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coef])
model2.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coef])
model3.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coef])
model4.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coef])

In [ ]:
images1 = get_images(4, SIZE_X, SIZE_Y)
masks1 = get_masks(4, SIZE_X, SIZE_Y)

In [ ]:
masks[0]

In [ ]:
# test_img2 = np.expand_dims(test_img2, axis=0).shape

In [ ]:
plt.figure(figsize=(6,3))
plt.imshow(images1[0])

In [ ]:
plt.figure(figsize=(6,3))
plt.imshow(masks1[0])

In [ ]:
prediction1 = model1.predict(images1)
prediction2 = model2.predict(images1)
prediction3 = model3.predict(images1)
prediction4 = model4.predict(images1)

In [ ]:
# prediction1 = predictions['1']
# prediction2 = predictions['2']
# prediction3 = predictions['3']
# prediction4 = predictions['4']

In [ ]:
# (prediction1[0]==prediction1[1]).sum()

In [ ]:
number_images = 5

random_index = np.round(np.random.rand(number_images) * len(prediction3)) + 1
print(random_index)

for i in range(number_images):
    # random_index = 'random_index_' + str(i+1)
    # print(random_index)
    # file_path_to_image = false_predicted_images['FilePath'][random_index[i]]
    # class_id = false_predicted_images['ClassId'][random_index[i]]
    # image_id = false_predicted_images['ImageId'][random_index[i]]
    # class_id_pred = int(false_predicted_images['ClassId_predicted'][random_index[i]])
    
    img = prediction3[int(random_index[i])]
    plt.figure(figsize=(18, 10))
    ax = plt.subplot(number_images, 1, i + 1)
    plt.imshow(img)
    # plt.title(f'Image ID: {image_id} | True ClassId: {class_id} | Predicted ClassId: {class_id_pred}', fontsize=16);
    plt.axis("off")

In [ ]:
# plt.figure(figsize=(16,3))
saved_model = False
class_id = 4

nrows = 20
ncols = 6
rows = list(range(nrows))
fig, axs = plt.subplots(nrows, ncols, figsize=(ncols*3,2*nrows))

# print(axs)

if saved_model:
    fig.suptitle(f'Performance of Mask Model from saved Model | Epochs: {EPOCHS}', fontsize=16)
else:
    fig.suptitle(f'Performance of Mask Model from fitted Model | Epochs: {EPOCHS}', fontsize=16)


for row, a, b, c, d, e, f in zip(rows, images1, masks1, prediction1, prediction2, prediction3, prediction4):
    # ax = plt.subplot(row, ncols, 1+(row-1)*6)
    axs[row, 0].imshow(a)
    # ax.set_axis_off()
    if row == 0:
        axs[row, 0].set_title(f'true image')
    axs[row, 0].set_ylabel(f'ClassId = {class_id}', fontsize=14)
    axs[row, 0].set_xticks([])
    axs[row, 0].set_yticks([])
    # plt.imshow(a)

    # ax = plt.subplot(row,ncols, 2)
    axs[row, 1].imshow(b)
    if row == 0:
        axs[row, 1].set_title('true mask')
    axs[row, 1].set_axis_off()
    
    # plt.imshow(b)

    # ax = plt.subplot(row,ncols, 3)
    axs[row, 2].imshow(c)
    if row == 0:
        axs[row, 2].set_title('mask model 1')
    axs[row, 2].set_axis_off()
    # plt.imshow(c)

    # ax = plt.subplot(row,ncols, 4)
    axs[row, 3].imshow(d)
    if row == 0:
        axs[row, 3].set_title('mask model 2')
    axs[row, 3].set_axis_off()
    # plt.imshow(d)

    # ax = plt.subplot(row,ncols, 5)
    axs[row, 4].imshow(e)
    if row == 0:
        axs[row, 4].set_title('mask model 3')
    axs[row, 4].set_axis_off()
    # plt.imshow(e)

    # ax = plt.subplot(row,ncols, 6)
    axs[row, 5].imshow(f)
    if row == 0:
        axs[row, 5].set_title('mask model 4')
    axs[row, 5].set_axis_off()
    # plt.imshow(f)
    #break
    if row == nrows:
        break

In [ ]:
np.unique(masks1[0])

In [ ]:
temp = get_masks(4,1600,256)

In [ ]:
np.unique(temp[1])